#Fe P speciation with AA calculated using PHREEQC

Using the default PHREEQC database file as a starting point the reactions of ascorbic acid were added.
Inputs include total iron, total phosphorus, total sulfur, total magnesium, total calcium, total inorganic carbon, total sodium, total chloride and total ascorbic acid, pH and pe

In [155]:
% setup environment
% set path to equilibrium solver
txt=pwd;
n=length(txt);
rootpath=txt(1:n-13);
PHREEQCpath=[rootpath,'runPHREEQC'];
addpath(PHREEQCpath);
% clear data and number format
clear; format short 

% turn off warnings
warning off

In [156]:
% input summary  -----------CHANGE THIS FOR YOUR SYSTEM --------
FeT=122.6; FeAW=55.847; FeT=(FeT*1e-3)/FeAW; 
PT=45.45; PAW=30.9738; PT=(PT*1e-3)/PAW;
ST=3.12; SAW=32.064; ST=(ST*1e-3)/SAW;
MgT=3.43; MgAW=34.312; MgT=(MgT*1e-3)/MgAW;
CaT=0.654; CaAW=40.08;CaT=(CaT*1e-3)/CaAW;
CT=42.86; CAW=12.0111;CT=(CT*1e-3)/CAW;
AAT=0.00448;
pH=4;
%pe= 0.422; 
Eh=+36; %mV
pe= (Eh*1e-3)/0.0592;
T=22;
NaT=82.1; NaAW=22.989; NaT=(NaT*1e-3)/NaAW; NaT=NaT+3*PT;
ClT=1.157; ClAW=35.453; ClT=(ClT*1e-3)/ClAW; ClT=ClT+3*FeT;

Format the inputs and define the outputs

In [157]:
minerals=[{' Fe(OH)3(a)'}; {'Vivianite'}; {'Calcite'}; {'Hydroxyapatite'}; {'Siderite'};{'FeSppt'};{'Pyrite'}; {'Strengite'} ];  
totalnames=[{'Fe(+3) '}; {'Fe(+2) '}; { 'Cl '}; { 'Na '}; { 'P '}; { 'Mg '}; {'Ca '}; {'S '}; { 'C(+4) '}; { 'D(-2)'}]; 
totalvector=[0              FeT          ClT       NaT       PT    MgT      CaT  ST     CT   AAT];
speciesexport=[{'HPO4-2'}; {'H2PO4-'}; {'H3PO4'}; {'PO4-3'}; {'MgHPO4'}; {'NaHPO4-'};{'CaH2PO4+'};{'MgH2PO4+'};{'CaHPO4'};{'CaPO4-'};{'MgPO4-'}];
speciesexport=[speciesexport; {'SO4-2'}; {'HS-'}; {'NaSO4-'}; {'MgSO4'};{'CaSO4'}];
speciesexport=[speciesexport; {'D-2'}; {'D'}; {'H2D'}; {'HD-'}; {'FeHD+'}; {'FeD+'}; {'FeOHD'}; {'FeD2-'}; {'Fe(OH)2D-'}; {'FeD'}];
speciesexport=[speciesexport; {'Hfo_wPO4-2'}; {'Hfo_wHPO4-'}; {'Hfo_wH2PO4'}];
speciesexport=[speciesexport;{'Fe+3'}; {'FeOH+2'}; {'Fe(OH)2+'}; {'Fe(OH)4-'}; {'FeHPO4+'}; {'FeH2PO4+2'}];
speciesexport=[speciesexport;{'Fe+2'}; {'FeHCO3+'}; {'FeHPO4'}; {'FeCO3'}; {'FeH2PO4+'}; {'FeSO4'}; {'FeCl+'}; {'FeOH+'}; {'FeHSO4+'}; {'Fe(HS)2'}; {'Fe(HS)3-'}];      
database=['PHREEQC_AA.dat']; show=0; %0 no output to screen. 1 output to screen. display does not work in Octave/Jupyter.  only in matlab version
acid=['HCl']; % need acid to maintain pH, can change which one.

Now put in the solver

In [158]:
[solutionspeciesconcs, solutionspeciesnames, solidconcs, solidnames]=...
    runPHREEQCv2noHA(T,pH,pe,totalnames,totalvector,minerals,speciesexport,database,show,acid);
    %convert phreeqc variable names for solids to matlab permissible names. set
    %solid concs as variable names
    for i=1:length(solidconcs)
         tst=cell2mat(solidnames(i));
         for j=1:length(tst)
             if tst(j)=='('; tst(j)='L'; end
             if tst(j)==')'; tst(j)='R'; end
             if tst(j)==':'; tst(j)='C'; end
             if tst(j)=='.'; tst(j)='p'; end
         end
         txt=[tst,'=solidconcs(i);']; eval(txt); % take out the semicolon after (i) if you want to see the solids listed with concs
    end
    for j=1:size(solutionspeciesconcs,1)
            name=cell2mat(solutionspeciesnames(j));
            %clean up name so it can be a matlab variable
            name=regexprep(name,'(mol/kgw)','');
            name=regexprep(name,'[m_]','');
            name = strrep(name,'+','plus');
            name = strrep(name,'-','minus');
            for k=1:length(name)
             if name(k)=='('; name(k)='L'; end
             if name(k)==')'; name(k)='R'; end
            end
            txt=[name,'=solutionspeciesconcs(j);'];
            eval(txt)
    end

%OUTPUTS
FeboundAA=FeHDplus+FeDplus+FeOHD+FeD2minus+FeLOHR2Dminus+FeD+FeD2minus;
percentFeAAbound=100*(FeboundAA/FeT)


ans = 0


percentFeAAbound = 6.7430
